In [ ]:
import numpy as np

seed = 1282
rng = np.random.RandomState(seed)

In [ ]:
from alignment import EPIC, distance as alignment_distance
from geom import orthogonal_project_planar, standardise, get_angle, random_heading_in_zero_mean_subspace, rotate_in_plane, random_epic_distance_step

### Testing standardisation

In [ ]:
a = rng.randn(5)
print(a.mean().round(3), np.linalg.norm(a).round(3))
s = standardise(a)
print(s.mean().round(3), np.linalg.norm(s).round(3))

### Testing orthogonal projection

In [ ]:
print(*orthogonal_project_planar([0, 0.001], rng.randn(2))) # (0, 1) and (+-1, 0)

In [ ]:
a, b = rng.randn(2, 5)
ao, bo = orthogonal_project_planar(a, b)
print('random norms and angle:', np.linalg.norm(a).round(3), np.linalg.norm(b).round(3), (get_angle(a, b) / np.pi).round(3))
print('orthonormalised norms and angle:', np.linalg.norm(ao).round(3), np.linalg.norm(bo).round(3), (get_angle(ao, bo) / np.pi).round(3))

### Testing rotation

In [ ]:
def random_zero_mean_rotation(v:np.ndarray, theta:float) -> np.ndarray:
    '''
    Randomly rotate given vector by given angle
    '''
    dim = v.shape[0]
    random_heading = random_heading_in_zero_mean_subspace(dim)
    return rotate_in_plane(v, random_heading, theta)

In [ ]:
# rotating a standardised vector by any amount should yield a standardised vector
a = standardise(rng.randn(5))
print('standard random mean, norm:', a.mean().round(3), np.linalg.norm(a).round(3))
u = random_zero_mean_rotation(a, rng.uniform(0, np.pi))
print('randomly rotated mean, norm:', u.mean().round(3), np.linalg.norm(u).round(3))

In [ ]:
# 2d
# specific rotations of known point
a = np.array([0, 1])
print(np.round(random_zero_mean_rotation(a, 0), decimals=2))     # 0, 1
print(np.round(random_zero_mean_rotation(a, np.pi), decimals=2))     # 0, -1
print(np.round(random_zero_mean_rotation(a, np.pi/2), decimals=2))   # +-1, 0
print(np.round(random_zero_mean_rotation(a, np.pi/3), decimals=2))   # +=sqrt(3/4), 1/2
print(np.round(random_zero_mean_rotation(a, np.pi/4), decimals=2))   # +=sqrt(1/2), sqrt(1/2)
print(np.round(random_zero_mean_rotation(a, np.pi/6), decimals=2))   # +-1/2, sqrt(3/4)
# random rotation should be by specified angle
theta = rng.uniform(0, np.pi)
a_rotated = random_zero_mean_rotation(a, theta)
theta_recovered = get_angle(a, a_rotated)
print('theta error:', round(abs(theta_recovered - theta), 2))

In [ ]:
# 10d
# random point, random rotations
a = rng.randn(10)
for i in range(10):
    theta = rng.uniform(0, np.pi)
    a_rotated = random_zero_mean_rotation(a, theta)
    theta_recovered = get_angle(a, a_rotated)
    print('theta error:', round(abs(theta_recovered - theta), 2))

In [ ]:
# rotating standardised vector should give angle-related distance
a = standardise(rng.randn(10))
theta = np.pi/3 # equilateral triangle
a_rotated = random_zero_mean_rotation(a, theta)
print(a.mean().round(3), np.linalg.norm(a).round(3))
print(a_rotated.mean().round(3), np.linalg.norm(a_rotated).round(3))
distance = np.linalg.norm(a - a_rotated)
print(distance.round(3)) # equilateral so distance should be 1

### Testing EPIC rotation

In [ ]:
for i in range(10):
    a = standardise(rng.randn(5))
    d_epic = rng.uniform(0, 1)
    a_stepped = random_epic_distance_step(a, d_epic)
    d_epic_recovered = alignment_distance(EPIC, a, a_stepped)
    print('EPIC error:', round(abs(d_epic_recovered - d_epic), 4), f'{a} by ({round(d_epic, 3)})')

In [ ]:
# 16d (4x4)
a = standardise(rng.randn(4, 4))
d_epic = rng.uniform(0, 1)
a_stepped = random_epic_distance_step(a, d_epic).reshape(a.shape)
d_epic_recovered = alignment_distance(EPIC, a, a_stepped)
print('EPIC error:', round(abs(d_epic_recovered - d_epic), 4))